# Question 1.

In [ ]:
# Importing numpy library
import numpy as np

def generate_input_data(N, M, D):
    """
    Function to generate input data matrix X for regression.

    Parameters in the function:
    N: Number of samples, is an integer.
    M: Number of columns, is an integer.
    D: Number of dimensions, is an integer.

    The function returns input data matrix X, which is an n-dimensional array.
    """

    # Generates random data matrix of size NxM using numpy
    sample_array = np.random.randn(N, M)

    if(M==D):
      # Generates an identity generatror-matrix of size MxM
      S = np.identity(M)
    else:
      # Generates an generator-matrix of size MxD
      S = np.random.rand(M, D)
    # Generates input data matrix of size NxD by matrix-multiplication of sample_array and S matrices.
    X = np.dot(sample_array, S)
    return X

# N = int(input("Input number of samples:")) # Takes number of samples as input from the user
# M = int(input("Input number of columns:")) # Takes number of columns as input from the user
# D = int(input("Input number of dimensions:")) # Takes number of dimensions as input from the user
N = 10000
M = 10
D = 5
X = generate_input_data(N, M, D)

print("Input data matrix X:")
print(X)


# Question 2

Used the formula:

$t = w^T x + ϵ (noise)$

$y = w^T x$

In [ ]:
sigma = 0.3 #Defining variance

w = np.ones((D+1,1)) #Initialising weight vector

def gen_target_vector(X, w, sigma):
  """
  The function generates target vector (Actual-Y values) with the inputs:
  X: input data matrix
  w: weight vector
  sigma: variance
  """
  t = np.dot(X, w) + np.random.normal(0, np.sqrt(sigma), size = (X.shape[0], 1)) # Generates the target vector
  return t

target = gen_target_vector(X, w[:-1], sigma)
print(target)

# Question 3.

In [ ]:
# Importing time library
import time

# Importing matplot library
import matplotlib.pyplot as plt

# Initialising an array to store the time taken to calculate the pseudo-inverse with different sample size inpur data matrix.
time_taken = []

for i in range(N):
    ith_sample_X = generate_input_data(i, M, 10)
    start_time = time.time()
    x_pseudo_inv = np.linalg.pinv(ith_sample_X)
    end_time = time.time()
    time_taken.append(end_time - start_time)

x = np.linspace(1, 10000, 10000)
plt.plot(x, time_taken,'o', markersize=1)
plt.xscale('log')
plt.yscale('log')
plt.title('Process Time vs. Sample Size')
plt.xlabel('Number of samples')
plt.ylabel('Time taken by the process')
plt.show()

# Question 4.

In [4]:
def nrmse(target_vector, y_pred):
    """
    This function returns the Noramalised Root Mean Squared Error using the input:
    target_vector: Th actual Y-values vector
    y_pred: The prediction matrix
    """
    rmse = np.sqrt(np.mean((target_vector - y_pred)**2)) # Calculate the Root Mean Sqared Error.

    range_values = np.max(target_vector) - np.min(target_vector) # Calculates the difference between the maximum and minimum values in target_vector.

    nrmse_value = rmse / range_values # Calculates the NMRSE value.

    return nrmse_value

# Question 5

In [ ]:
def gradient_mean_square_error(X, w, true_vec):
    """
    The function generates the graident of the Mean Square Error using inputs:
    X: input data matrix
    w: weight vector
    true_vec: true observed values (randomly generated but fixed)
    """
    # Calculates the predicted vector using X and w.
    pred_vec = np.dot(X, w)
    # Calculates the gradient of the Mean Square Error
    gradient = -2*np.dot(X.T, (true_vec - pred_vec))/len(pred_vec)

    # Returns the gradient of the Mean Square Error.
    return gradient


grad_mse = gradient_mean_square_error(X, w[:-1], target) # Calculating the gradient of the Mean Square Error

print(f"Gradient of Mean Square Error (MSE): {grad_mse}") #Printing the gradient value
print(grad_mse.shape)

# Question 6.

In [6]:
def L2_norm(w):
    """
    This function returns the L2 norm with the inputs:
    w: weight vector
    """
    norm_L2 = np.linalg.norm(w, ord = 2) # Calculates the norm using the function np.linalg.norm.

    grad_L2 = w/norm_L2
    return grad_L2




# Question 7

In [7]:
def L1_norm(w):
    """
    This function returns the L1 norm with the inputs:
    w: weight vector
    """
    norm_L1 = np.linalg.norm(w, ord = 1) # Calculates the norm using the function np.linalg.norm.

    grad_L1 = w/norm_L1
    return grad_L1


# Question 8

In [8]:
def mse(pred_y,target_vector):
  """
  This function calculates the Mean Squared Error using the inputs:
  pred_y: the predicted values matrix
  target_vector: the matrix containing the actual Y-values
  """
  # print(pred_y.shape, target_vector.shape)
  mse = np.mean((target_vector - pred_y)**2)
  return mse

In [9]:
def gradient_descent_wts(x, t, w, alpha, lambda_1, lambda_2):
  """
  This function returns the updated weights vector using gradient descent and updated Mean Sqaured Error using the mse function defined above, with the inputs:
  x: input data matrix
  t: target vector containing the actual Y-values
  w: weights vector
  alpha: learning rate
  lambda1 and lambda2: regularisation parameter
  """
  wts_new = w - alpha * (gradient_mean_square_error(x, w, t) + lambda_1 * L1_norm(w) + lambda_2 * L2_norm(w)) # This line of code updates the weights vector.

  y = np.dot(x, wts_new) # Calculates the new Y matrix using the updated weights.

  mse_new = mse(y,t) # Calculates the updated MSE using the updated Y matrix.
  # print(mse)
  #print(eta * (gradient_mean_square_error(x, w, t) + lambda_1 * L1_norm(w) + lambda_2 * L2_norm(w)))
  return wts_new, mse_new

In [10]:
def grad_descent(x,t,w,eta,lambda_1, lambda_2, max_iter, min_change_MSE):
  """
  This function returns the updated weights vector and updated MSE value after iterating it through multiple times to get the MSE error below
  a threshold value which implies a close-to-zero slope using the inputs:
  x: input data matrix
  t: target vector containing the actual Y-values
  w: weights vector
  alpha: learning rate
  lambda1 and lambda2: regularisation parameter
  max_iter: maximum number of iterations
  min_change_MSE: the threshold MSE value below which the loop stops and we reach a close-to-zero slope.
  """
  mse_new = 100 # Dummy MSE

  wts_new = w.copy() # Makes a copy of the updated weights after every iteration

  mse_old = 0 # Dummy MSE

  diff = 0 # Holds the difference between the mse_new and mse_old.

  for i in range(max_iter): # Loops around maximum iterations
    if diff <= min_change_MSE and i > 1: # Stops loop if difference between last and current NRMSE is less than the threshold value.
      break
    p_new = gradient_descent_wts(x, t, wts_new, eta, lambda_1, lambda_2)
    wts_new = p_new[0]
    mse_new = p_new[1]

    diff = abs(mse_new - mse_old)

    mse_old = mse_new

  return wts_new, mse_new

# Question 9.

In [11]:
def NRMSE(x, t, w, eta, lambda_1, lambda_2, max_iter, min_change_NRMSE):
  """
  This function returns the updated weights vector and updated MSE value after iterating it through multiple times to get the NRMSE error below
  a threshold value which implies a close-to-zero slope using the inputs:
  x: input data matrix
  t: target vector containing the actual Y-values
  w: weights vector
  alpha: learning rate
  lambda1 and lambda2: regularisation parameter
  max_iter: maximum number of iterations
  min_change_NRMSE: the threshold NRMSE value below which the loop stops and we reach a close-to-zero slope.

  """
  mse_new = 100 # Dummy

  wts_new = w.copy() # Makes a copy of the updated weights after every iteration

  std_t = np.std(t) # Standard deviation for target matrix, for NRMSE calculation

  nrmse_old = 0

  diff = 0 # Holds the difference between the nrmse_new and nrmse_old.

  for i in range(max_iter) :# Loops around maximum iterations
    if diff <= min_change_NRMSE and i > 1: # Stops loop if difference between last and current NRMSE is less than the threshold value.
      break
    final_wts, mse_new = gradient_descent_wts(x,t,wts_new,eta,lambda_1, lambda_2)

    nrmse_new = np.sqrt(mse_new) / std_t
    diff = abs(nrmse_new - nrmse_old)
    nrmse_old = nrmse_new


  return final_wts, nrmse_new

In [ ]:
# IMporting the train_test_split function to split the input sample into trainging data and test data.
from sklearn.model_selection import train_test_split

sigma = [0.5, 1, 1.5, 2, 2.5] # Stores the various variance values
sigma_impact = [] # Stores the change in

for i in sigma:
  target_i = gen_target_vector(X, w[:-1], i)

  X_train, X_test, t_train, t_test = train_test_split(X, target_i, test_size=0.33, random_state=42)

  wts_new, nrmse = NRMSE(X_train, t_train, w[:-1], 1e-2, 1e-1, 1e-1, 10000,1e-5)

  sigma_impact.append(nrmse)

for i in range(len(sigma)):
  print(f"When variance is {sigma[i]}, the value of NRMSE is: {sigma_impact[i]}")

In [ ]:
# PLotting the graph between Variance vs. value of NRMSE.
plt.plot(sigma, sigma_impact)
plt.title('Impact of sigma on NRMSE')
plt.xlabel('sigma')
plt.ylabel('Value of NRMSE after test')
plt.show()

# Question 10.

In [ ]:
N_values = [10, 100, 1000, 10000, 100000] # Array initialising the number of samples values.

lambda2_values = [0.001,0.01,0.1,1.0,10.0] # Array initialising the lambda2 values.

sigma = 0.3

# The below loop stores the averaged out NRMSE values over 5 iterations for various N and sigma pairs.
for i in range(5):
  nmrse = 0
  for j in range(5):
    w = np.ones((D+1,1))
    x_q10 = generate_input_data(N_values[j], M, D)
    t_q10 = gen_target_vector(x_q10, w[:-1], sigma)
    t, t1 = NRMSE(x_q10, t_q10, w[:-1], 1e-2, 1e-1, lambda2_values[i], 10000,1e-5)
    nmrse += t1

  print(f"For lambda2 values {lambda2_values[i]}, the averaged out NRMSE values is {nmrse/5}")

# Question 11.

In [ ]:
lambda1_values = np.array([0.001, 0.01, 0.1, 1.0, 10.0]) #  Array initialising the lambda2 values.

# Defining the correlation matrix.
correlation_matrix = np.array([[1, 0.8, 0.5, 0.2, 0.1],
                               [0.8, 1, 0.7, 0.3, 0.2],
                               [0.5, 0.7, 1, 0.4, 0.3],
                               [0.2, 0.3, 0.4, 1, 0.6],
                               [0.1, 0.2, 0.3, 0.6, 1]])


weights_list = []
for i in range(5):
    w = np.ones((D+1, 1))
    x_q11 = generate_input_data(N, M, D)
    t_q11 = gen_target_vector(x_q11, w[:-1], sigma)

    # Transform X to make it correlated using the Cholesky decomposition.
    X_correlated = X.dot(np.linalg.cholesky(correlation_matrix).T)
    t, t1 = grad_descent(X_correlated, t_q11, w[:-1], 1e-2, lambda1_values, 1e-1, 10000,1e-5)

    weights_list.append(t)

# print(f"Shape of x: {lambda1_values.shape}")
# print(f"Shape of y: {weights_list.shape}")


weights_array = np.array(weights_list)
# weights_array = weights_array.reshape(-1,)

# lambda1_values = lambda1_values.reshape((5, 1))
for i in range(weights_array.shape[1]):
    plt.plot(1 / lambda1_values, weights_array[:, i], label=f'Weight {i+1}', marker = 'o')
    # plt.plot([1 / lambda1 for lambda1 in lambda1_values], [weights[i] for weights in weights_list], label=f'Weight {i+1}')
    # plt.plot(1 / lambda1_values, weights_array[:, i], label=f'Weight {i+1}')

plt.xscale('log')  # Set x-axis to log scale
plt.xlabel('1/lambda1')
plt.ylabel('Weights')
plt.title('Weights vs. 1/lambda for Linear Regression')
plt.grid()
plt.show()

# Question 12.

In [ ]:
# Importing elastic net and standard scaler function from sklearn.
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler

xx = generate_input_data(N, M, D)

tt = gen_target_vector(x_q11, w[:-1], sigma)

scaler = StandardScaler()
X_standardized = scaler.fit_transform(xx)

# Apply Elastic Net with varying alpha values
alpha_values = [0.1, 0.63, 0.55, 0.87]
weights_list = []

for alpha in alpha_values:
    elastic_net = ElasticNet(alpha=1, l1_ratio = alpha)
    elastic_net.fit(xx, tt)
    weights_list.append(elastic_net.coef_)

# Convert to numpy array for easier manipulation
weights_array = np.array(weights_list)

# Plotting
plt.figure(figsize=(10, 6))
for i in range(weights_array.shape[1]):
    plt.plot(weights_array[:, i], label=f'Feature {i+1}')

plt.xlabel('Alpha (Mixing parameter)')
plt.ylabel('Weights')
plt.title('Grouping Effect of Elastic Net on Correlated Columns')
plt.legend()
plt.show()

# Question 13.


In [ ]:
def linear_bin_class(X, w, b, sigma):
  """
  This funcion returns the binary classification array using the inputs:
  X: input data matrix
  w: weights vectos
  b: bias
  sigma: variance
  """
  linear_part = np.dot(X, w) + b # Calculates the linear part of the prediction matrix.

  t = np.sign(linear_part + np.random.normal(0, np.sqrt(sigma), size = (X.shape[0], 1))) #Calculates the target vector

  return t

X = generate_input_data(N, M ,D)
w = np.ones((D+1, 1))
b = 0.2
vec = linear_bin_class(X, w[:-1], b, sigma)

print(vec)


# Question 14.

In [ ]:
def sigmoid_func(z):
  """
  This function returns the sigmoid values using the input:
  z: prediction matrix
  """

  return 1/(1 + np.exp(-z))

def cross_entropy(X, target_vector, w, b):
  """
  This function returns the gradients of weights and bias using inputs:
  X: input data matrix
  target_vector: vector containing actual Y-values
  w: weights vector
  b: bias
  """

  m = len(target_vector)

  pred_y = sigmoid_func(X.dot(w) + b)

  weight_gradients = X.T.dot(pred_y - target_vector)/m

  gradient_bias = np.sum(pred_y - target_vector)/m

  return weight_gradients, gradient_bias


X = generate_input_data(N, M, D)

target_vector = gen_target_vector(X, w[:-1], sigma)

bias = 2

gradient_of_weights, gradient_bias = cross_entropy(X, target_vector, w[:-1], bias)

print(f"Gradient of weights: {gradient_of_weights}")
print(f"Gradient of bias: {gradient_bias}")

In [19]:
def q15(x,t,w,eta,labda_1, labda_2, max_iter, min_change_NRMSE):
  """
  This function returns the updated weights vector and updated MSE value after iterating it through multiple times to get the NRMSE error below
  a threshold value which implies a close-to-zero slope using the inputs:
  x: input data matrix
  t: target vector containing the actual Y-values
  w: weights vector
  alpha: learning rate
  lambda1 and lambda2: regularisation parameter
  max_iter: maximum number of iterations
  min_change_NRMSE: the threshold NRMSE value below which the loop stops and we reach a close-to-zero slope.
  """
  mse_new = 100 # Dummy
  wts_new = w.copy() # Makes a copy of the updated weights after every iteration.
  std_t = np.std(t) # Standard deviation for t matrix, for nrmse calculation.
  nrmse_old = 0
  diff = 0

  for i in range(max_iter): # Loops around for maximum iterations
    if diff <= min_change_NRMSE and i > 1: # s\Stops loop if difference between last and current NRMSE is less than the threshold value.
      break
    final_wts, mse_new = cross_entropy(X, target_vector, w[:-1], bias)
    nrmse_new = np.sqrt(mse_new) / std_t
    diff = abs(nrmse_new - nrmse_old)
    nrmse_old = nrmse_new


  return final_wts, nrmse_new

# Question 15.

In [ ]:
# Repeats question 10 using binary classification.
N_values = [10, 100, 1000, 10000, 100000]

lambda2_values = [0.001,0.01,0.1,1.0,10.0]

sigma = 0.3


for i in range(5):
  nmrse = 0
  for j in range(5):
    w = np.ones((D+1,1))
    x_q10 = generate_input_data(N_values[j], M, D)
    t_q10 = gen_target_vector(x_q10, w[:-1], sigma)

    t, t1 = q15(x_q10, t_q10, w, 1e-2, 1e-1, lambda2_values[i], 10000,1e-5)
    nmrse += t1


  print(f"For lambda2 values {lambda2_values[i]}, the averaged out NRMSE values is {nmrse/5}")

The only help I took was from ChatGPT, everything else was done solely by me.